1- Préparation des données

In [ ]:
import pandas as pd 
import numpy as np
 
#chargement de la base de données
df= pd.read_csv("C:/Users/wague/Downloads/projetia/clean_IA.csv", parse_dates=['BaseDateTime'])
#parse_dates permet de convertir la colonne BaseDateTime en format datetime


#Tri des données par date et par MMSI
df = df.sort_values(by=['MMSI','BaseDateTime'])

#Variables importantes: LAT,LON,SOG,COG, Heading, VesselType, Length, Draft
#Variables dépendantes à prédire: LAT et LON
targets = ['LAT', 'LON']
#Variables explicatives: SOG, COG, Heading, potentiellement enrichi par: VesselType, Length, Draft
features = ['SOG', 'COG', 'Heading']
#Affichage des 15 premières lignes
print(df.head(15))

In [50]:
#Afin de prédire la position du navire toute les 5 minutes
period=5
x,y_5,y_10,y_15 = [], [], [], []
for mmsi, group in df.groupby('MMSI'):
    #On récupère les données du navire
    group = group.sort_values(by='BaseDateTime')
    
    for i in range(len(group) - period -15): #Pourquoi -15?
    #On récupère les données de la période de 5 minutes
        x_period = group.iloc[i:i + period-1][features].values #Pourquoi -1?
        y_5_period = group.iloc[i+5][features].values
        y_10_period = group.iloc[i+10][features].values
        y_15_period = group.iloc[i+15][features].values
        #On ajoute les valeurs à la liste globale
        x.append(x_period)
        y_5.append(y_5_period)
        y_10.append(y_10_period)
        y_15.append(y_15_period)

x = np.array(x)
y_5 = np.array(y_5)
y_10 = np.array(y_10)
y_15 = np.array(y_15)
